# 1. Download data

In [1]:
# pip install beautifulsoup4
# pip install lxml
# pip install geopy

import bs4 as bs
import urllib.request
import pandas as pd 
import numpy as np 

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')

print('Libraries imported.')

Libraries imported.


### extract data

In [2]:
table = soup.find('table')
table_rows = table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

import pandas as pd

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
for df in dfs:
    print(df)


[]
['M1A\n', 'Not assigned\n', 'Not assigned\n']
['M2A\n', 'Not assigned\n', 'Not assigned\n']
['M3A\n', 'North York\n', 'Parkwoods\n']
['M4A\n', 'North York\n', 'Victoria Village\n']
['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n']
['M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n']
['M7A\n', 'Downtown Toronto\n', "Queen's Park, Ontario Provincial Government\n"]
['M8A\n', 'Not assigned\n', 'Not assigned\n']
['M9A\n', 'Etobicoke\n', 'Islington Avenue, Humber Valley Village\n']
['M1B\n', 'Scarborough\n', 'Malvern, Rouge\n']
['M2B\n', 'Not assigned\n', 'Not assigned\n']
['M3B\n', 'North York\n', 'Don Mills\n']
['M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n']
['M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n']
['M6B\n', 'North York\n', 'Glencairn\n']
['M7B\n', 'Not assigned\n', 'Not assigned\n']
['M8B\n', 'Not assigned\n', 'Not assigned\n']
['M9B\n', 'Etobicoke\n', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n']


### Ignore cells with a borough that is Not assigned; More than one neighborhood can exist in one postal code area

In [3]:
dft = dfs[0]
dft2 = dft[~dft.Borough.str.contains("Not assigned")]
dft2 = dft2.rename(columns = {'Postal Code':'PostalCode'}).reset_index(drop=True)
dft2.groupby('PostalCode')

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
mask = dft2['Neighborhood'] == "Not assigned"
dft2.loc[mask, 'Neighborhood'] = dft2.loc[mask, 'Borough']
dft2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
dft2.shape

(103, 3)

# 2. Using the .csv file to join the geospatial data to the PostalCode file

In [6]:
gis = pd.read_csv("http://cocl.us/Geospatial_data")
gis = gis.rename(columns = {'Postal Code':'PostalCode'}).reset_index(drop=True)
gis.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
dfn = pd.merge(dft2,gis)

In [8]:
dfn.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# 3. Clustering & Mapping Toronto

In [9]:
# pip install requests 
# pip install  matplotlib
# pip install  scikit-learn
# pip install folium
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfn['Borough'].unique()),
        dfn.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [11]:
CLIENT_ID = 'JTBDLVXOVXOMK5C2CIJZVEL4XLRNIIHOYFSAWIVIKA5DXSBV' # your Foursquare ID
CLIENT_SECRET = 'EE2RCSEVPT21M33MIWBNUDLK23VVPGJEDOQ5VF5EUMEKRDLE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JTBDLVXOVXOMK5C2CIJZVEL4XLRNIIHOYFSAWIVIKA5DXSBV
CLIENT_SECRET:EE2RCSEVPT21M33MIWBNUDLK23VVPGJEDOQ5VF5EUMEKRDLE


### create a function to repeat the nearby venues to all the neighborhoods in Manhattan

In [12]:
radius=500
LIMIT = 30
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### create a new dataframe called to_venues

In [13]:
to_venues = getNearbyVenues(names=dfn['Neighborhood'],
                                   latitudes=dfn['Latitude'],
                                   longitudes=dfn['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [14]:
print(to_venues.shape)
to_venues.head()
to_venues.groupby('Neighborhood').count().head() 

(1333, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24


In [15]:
print('There are {} uniques categories.'.format(len(to_venues['Venue Category'].unique())))

There are 234 uniques categories.


### Analyze Each Neighborhood

In [16]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhood'] = to_venues['Neighborhood'] 

# moving neighborhood to the first column
col_name="Neighborhood"
first_col = to_onehot.pop(col_name)
to_onehot.insert(0, col_name, first_col)
to_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
to_onehot.shape

(1333, 234)

### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
to_grouped = to_onehot.groupby('Neighborhood').mean().reset_index()
to_grouped.head(3)

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### create the new dataframe and display the top 10 venues for each neighborhood

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = to_grouped['Neighborhood']

for ind in np.arange(to_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Pub,Sandwich Place,Pharmacy,Gym,General Travel,Curling Ice,Donut Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Supermarket,Bridal Shop,Shopping Mall,Sandwich Place,Restaurant,Pizza Place,Mobile Phone Shop
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Yoga Studio,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Sushi Restaurant,Pharmacy,Thai Restaurant,Juice Bar,Butcher,Café


# cluster

In [20]:
# set number of clusters
kclusters = 5

to_grouped_clustering = to_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = dfn.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

to_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Bus Stop,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Yoga Studio,Historic Site,Pub,Performing Arts Venue,Mexican Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Miscellaneous Shop,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,Park


# mapping & cluster figure

In [21]:
# drop to_merged NaN rows
to_merged = to_merged.dropna(axis='rows')

# create map
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### cluster table

In [22]:
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,0.0,Playground,Yoga Studio,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
85,Scarborough,0.0,Playground,Park,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
91,Downtown Toronto,0.0,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [23]:
to_merged.loc[to_merged['Cluster Labels'] == 1, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
2,Downtown Toronto,1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Yoga Studio,Historic Site,Pub,Performing Arts Venue,Mexican Restaurant
3,North York,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Miscellaneous Shop,Dessert Shop,Electronics Store
4,Downtown Toronto,1.0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,Park
6,Scarborough,1.0,Fast Food Restaurant,Department Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,1.0,Bakery,Café,Coffee Shop,Restaurant,Italian Restaurant,Playground,Jewelry Store,Caribbean Restaurant,Japanese Restaurant,Indian Restaurant
97,Downtown Toronto,1.0,Café,Restaurant,Coffee Shop,Seafood Restaurant,Hotel,Tea Room,Gluten-free Restaurant,Bookstore,Speakeasy,Pub
99,Downtown Toronto,1.0,Coffee Shop,Gay Bar,Ramen Restaurant,Bookstore,Juice Bar,Breakfast Spot,Bubble Tea Shop,Burger Joint,Salon / Barbershop,Diner
100,East Toronto,1.0,Light Rail Station,Gym / Fitness Center,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Skate Park


In [24]:
to_merged.loc[to_merged['Cluster Labels'] == 2, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2.0,Baseball Field,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
101,Etobicoke,2.0,Baseball Field,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [25]:
to_merged.loc[to_merged['Cluster Labels'] == 3, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Etobicoke,3.0,River,Yoga Studio,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [26]:
to_merged.loc[to_merged['Cluster Labels'] == 4, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Park,Food & Drink Shop,Bus Stop,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
12,Scarborough,4.0,Bar,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
21,York,4.0,Park,Women's Store,Convenience Store,Bar,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
35,East York,4.0,Park,Convenience Store,Intersection,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
61,Central Toronto,4.0,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
64,York,4.0,Park,Convenience Store,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
66,North York,4.0,Park,Convenience Store,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop


### five clusters were made, cluster 0 were recreation places; cluster 1 were eating areas; cluster 2 were sports areas; cluster 3 was a riverbank area; cluster 4 were park area